<a href="https://colab.research.google.com/github/Richard-4/UASDL/blob/main/UAS_no_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate

In [27]:
import pandas as pd
import numpy as numpy

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler

from tqdm.auto import tqdm

from datasets import Dataset

import evaluate

In [28]:
emotion = pd.read_csv('/content/Emotion.csv', sep=',')

emotion.head()

,text,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [29]:
emotion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    20000 non-null  object
 1   label   20000 non-null  object
dtypes: object(2)
memory usage: 312.6+ KB


In [30]:
emotion.rename(columns={'label':'labels'}, inplace=True)

In [31]:
emotion.head()

,text,labels
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [32]:
emotion['labels'].value_counts()

joy         6761
sadness     5797
anger       2709
fear        2373
love        1641
surprise     719
Name: labels, dtype: int64

In [33]:
emotion['labels'] = emotion['labels'].map({'joy': 0, 'sadness' : 1, 'anger' : 2, 'fear' : 3, 'love' : 4, 'surprise' : 5})

In [34]:
emotion.head()

,text,labels
0,i didnt feel humiliated,1
1,i can go from feeling so hopeless to so damned...,1
2,im grabbing a minute to post i feel greedy wrong,2
3,i am ever feeling nostalgic about the fireplac...,4
4,i am feeling grouchy,2


In [35]:
import re
import unicodedata
import nltk
from nltk.corpus import stopwords

def character_encoding(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def remove_stopwords(w):
    stopwords_list=stopwords.words('english')
    words = w.split()
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words)

def clean_sentence(w):
    #w = character_encoding(w.lower().strip())
    #w = re.sub(r"([?.!,¿])", r" ", w)
    #w = re.sub(r'[" "]+', " ", w)
    #w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w=remove_stopwords(w)
    #w=re.sub(r'@\w+', '',w)
    return w

nltk.download('stopwords')

emotion['text']= emotion['text'].map(clean_sentence)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
train_data = emotion.sample(frac=0.7, random_state=42)

test_data = emotion.drop(train_data.index)

validation_data = test_data.sample(frac=0.5, random_state=42)

test_data = test_data.drop(validation_data.index)

print("Train data =", len(train_data))
print("Test data =", len(test_data))
print("Validation data =", len(validation_data))

Train data = 14000
Test data = 3000
Validation data = 3000


In [37]:
hg_train = Dataset.from_pandas(train_data)
hg_test = Dataset.from_pandas(test_data)
hg_validation = Dataset.from_pandas(validation_data)

In [38]:
tokenizer = AutoTokenizer.from_pretrained("nghuyong/ernie-2.0-large-en")

In [39]:
tokenizer

BertTokenizerFast(name_or_path='nghuyong/ernie-2.0-large-en', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [40]:
def tokenize_dataset(data):
  return tokenizer(data["text"],
                   max_length = 32,
                   truncation = True,
                   padding = "max_length")

In [41]:
train = hg_train.map(tokenize_dataset)
test = hg_test.map(tokenize_dataset)
validation = hg_validation.map(tokenize_dataset)

Map:   0%|          | 0/14000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [42]:
print(test)
print(train)
print(validation)

Dataset({
    features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3000
})
Dataset({
    features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 14000
})
Dataset({
    features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3000
})


In [43]:
train = train.remove_columns(["text","__index_level_0__"])
test = test.remove_columns(["text","__index_level_0__"])
validation = validation.remove_columns(["text","__index_level_0__"])

In [44]:
train.set_format("torch")
test.set_format("torch")
validation.set_format("torch")

In [45]:
print(train)

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 14000
})


In [46]:
train[0]

{'labels': tensor(2),
 'input_ids': tensor([  101,  4384,  2195,  2706,  3283,  2707,  3110, 24501,  4765,  3993,
          2939,  2646, 14662,  5153,  6808,  2607,  5153,  2346,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0])}

In [47]:
torch.cuda.empty_cache()

train_dataloader = DataLoader(dataset=train, shuffle=True, batch_size = 10)
test_dataloader = DataLoader(dataset=test, batch_size = 4)
validation_dataloader = DataLoader(dataset=validation, batch_size = 4)

In [48]:
model = AutoModelForSequenceClassification.from_pretrained("nghuyong/ernie-2.0-large-en", num_labels=6)

Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-2.0-large-en and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
epochs = 5

training_steps = epochs * len(train_dataloader)

optimizer = AdamW(params=model.parameters(), lr=5e-6)

lr_scheduler = get_scheduler(name="linear",
                             optimizer=optimizer,
                             num_warmup_steps=0,
                             num_training_steps=training_steps)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

ErnieForSequenceClassification(
  (ernie): ErnieModel(
    (embeddings): ErnieEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(4, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ErnieEncoder(
      (layer): ModuleList(
        (0-23): 24 x ErnieLayer(
          (attention): ErnieAttention(
            (self): ErnieSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ErnieSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNo

In [50]:
progress = tqdm(range(training_steps))
model.train()

for epoch in range(epochs):
  for batch in train_dataloader:
    batch = {key: val.to(device) for key, val in batch.items()}
    outputs = model (**batch)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress.update(1)

  0%|          | 0/7000 [00:00<?, ?it/s]

In [52]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
recall = evaluate.load("recall")

In [63]:
model.eval()

val_loss = 0.0
correct_preds = 0.0
total_preds = 0.0

In [67]:
for batch in validation_dataloader:
  batch = {key: val.to(device) for key, val in batch.items()}
  labels = batch['labels'].to(device)
  with torch.no_grad():
    outputs = model (**batch)
  loss = outputs.loss
  val_loss += loss.item()

  logits = outputs.logits
  _, preds = torch.max(logits, dim=1)
  correct_preds += torch.sum(preds == labels).item()
  total_preds += len(labels)

val_accuracy = correct_preds / total_preds
print(f"Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")


Loss: 115.3648, Validation Accuracy: 0.9287


In [75]:
model.eval()

total_preds, total_correct, total_loss = 0, 0, 0
all_preds = []
all_labels = []

In [80]:
for batch in test_dataloader:
  batch = {key: val.to(device) for key, val in batch.items()}
  labels = batch['labels'].to(device)
  with torch.no_grad():
    outputs = model (**batch)
  loss = outputs.loss
  logits = outputs.logits

  total_loss += loss.item()
  _, preds = torch.max(logits, dim=1)
  total_correct += torch.sum(preds == labels).item()
  total_preds += len(labels)

  all_preds.extend(preds.cpu().numpy())
  all_labels.extend(preds.cpu().numpy())

In [81]:
average_loss = total_loss / len(test_dataloader)
accuracy = total_correct / total_preds

In [82]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')


print(f"Test Loss: {average_loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

Test Loss: 0.2960
Test Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000
